In [2]:
from pathlib import Path
import sys
import pandas as pd
import plotly.io as pio
import plotly.express as px



ROOT = Path("C:/Users/Marei/project/bootcamp/week2-data-work")
sys.path.insert(0, str(ROOT / "src"))

from bootcamp_data.config import make_paths

p = make_paths(ROOT)

df  = pd.read_parquet(
    p.processed / "analytics_table.parquet",
    engine="fastparquet"
)


In [3]:
DATA = ROOT / "data" / "processed" / "analytics_table.parquet"

FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)


In [4]:
df = pd.read_parquet(DATA, engine="fastparquet")
print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))

missing = df.isna().sum().sort_values(ascending=False).head(10)
print(missing)




rows: 5 cols: 17
order_id                      object
user_id                       object
amount                       float64
quantity                       Int64
created_at       datetime64[ns, UTC]
status                        object
status_clean                  object
amount_isna                     bool
quantity_isna                   bool
date                          object
year                         float64
month                         object
dow                           object
hour                         float64
country                       object
dtype: object
amount           1
amount_winsor    1
quantity         1
created_at       1
hour             1
dow              1
month            1
year             1
date             1
order_id         0
dtype: int64


# EDA Questions

1. What is the distribution of total sales across product categories?
2. Which products have the highest and lowest total sales?
3. How do monthly sales change over time?
4. Which customers contribute the most to total revenue?


In [5]:
rev_country = (
    df.groupby("country", dropna=False)
    .agg(
        n_orders=("order_id", "size"),
        revenue=("amount", "sum"),
        aov=("amount", "mean")
    )
    .reset_index()
    .sort_values("revenue", ascending=False)
)

fig1 = px.bar(
    rev_country,
    x="country",
    y="revenue",
    title="Revenue by Country"
)

ROOT = Path.cwd().parent
p = make_paths(ROOT)

fig1.write_image(p.figures / "revenue_by_country.png")
print("Figure saved to:", p.figures / "revenue_by_country.png")


Figure saved to: c:\Users\Marei\project\bootcamp\week2-data-work\reports\figures\revenue_by_country.png


In [6]:
sales_by_customer = (
    df[df["status_clean"] == "paid"]
    .groupby("user_id")
    .agg(total_sales=("amount_winsor", "sum"),
         n_orders=("order_id", "count"))
    .reset_index()
    .sort_values("total_sales", ascending=False)
)

top_customers = sales_by_customer.head(10)
bottom_customers = sales_by_customer.tail(10)

fig_top = px.bar(
    top_customers,
    x="user_id",
    y="total_sales",
    title="Top 10 Customers by Total Sales"
)

fig_bottom = px.bar(
    bottom_customers,
    x="user_id",
    y="total_sales",
    title="Bottom 10 Customers by Total Sales"
)

fig_top.write_image(p.reports / "figures" / "top_customers.png")
fig_bottom.write_image(p.reports / "figures" / "bottom_customers.png")




In [7]:
monthly_sales = (
    df[(df["status_clean"] == "paid") & (df["month"].notna())]
    .groupby("month")
    .agg(total_sales=("amount_winsor", "sum"))
    .reset_index()
    .sort_values("month")
)

fig = px.line(
    monthly_sales,
    x="month",
    y="total_sales",
    title="Monthly Sales Trend",
    markers=True
)

fig.write_image(p.reports / "figures" / "monthly_sales_trend.png")


In [8]:
customer_revenue = (
    df[df["status_clean"] == "paid"]
    .groupby("user_id")
    .agg(total_revenue=("amount_winsor", "sum"))
    .reset_index()
    .sort_values("total_revenue", ascending=False)
)

customer_revenue["revenue_share"] = (
    customer_revenue["total_revenue"]
    / customer_revenue["total_revenue"].sum()
)

customer_revenue["cumulative_share"] = customer_revenue["revenue_share"].cumsum()

fig = px.line(
    customer_revenue,
    y="cumulative_share",
    title="Cumulative Revenue Contribution by Customers"
)

fig.write_image(p.reports / "figures" / "customer_revenue_pareto.png")



In [9]:
monthly_table = (
    df[df["status_clean"] == "paid"]
    .dropna(subset=["month"])
    .groupby("month")
    .agg(
        n=("order_id", "count"),
        revenue=("amount_winsor", "sum")
    )
    .reset_index()
    .sort_values("month")   
)

monthly_table

,month,n,revenue
0,2025-12,3,45.635


In [ ]:


ROOT = Path.cwd().parent
p = make_paths(ROOT)


df_paid = df[
    (df["status_clean"] == "paid") &
    (df["amount_winsor"].notna())
]


fig = px.histogram(
    df_paid,
    x="amount_winsor",
    nbins=30,
    title="Distribution of Order Amounts (Winsorized)",
    labels={"amount_winsor": "Order Amount"}
)


fig.write_image(p.reports / "figures" / "amount_hist_winsor.png")



In [11]:
import numpy as np


df_paid = df[
    (df["status_clean"] == "paid") &
    (df["amount_winsor"].notna())
]

group_sa = df_paid[df_paid["country"] == "SA"]["amount_winsor"].values
group_non_sa = df_paid[df_paid["country"] != "SA"]["amount_winsor"].values

n_sa = len(group_sa)
n_non_sa = len(group_non_sa)

print("Sample sizes:")
print("SA:", n_sa)
print("Non-SA:", n_non_sa)

diff_mean = group_sa.mean() - group_non_sa.mean()
print("Observed diff_mean (SA - Non-SA):", diff_mean)

rng = np.random.default_rng(42)

B = 10_000
boot_diffs = np.empty(B)

for i in range(B):
    sa_sample = rng.choice(group_sa, size=n_sa, replace=True)
    non_sa_sample = rng.choice(group_non_sa, size=n_non_sa, replace=True)
    boot_diffs[i] = sa_sample.mean() - non_sa_sample.mean()

ci_low, ci_high = np.percentile(boot_diffs, [2.5, 97.5])

print("95% Bootstrap CI:")
print("CI low :", ci_low)
print("CI high:", ci_high)


Sample sizes:
SA: 4
Non-SA: 0
Observed diff_mean (SA - Non-SA): nan


C:\Users\Marei\AppData\Local\Temp\ipykernel_10980\1839039920.py:19: RuntimeWarning:

Mean of empty slice

c:\Users\Marei\project\bootcamp\week2-data-work\.venv\Lib\site-packages\numpy\_core\_methods.py:142: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Marei\AppData\Local\Temp\ipykernel_10980\1839039920.py:30: RuntimeWarning:

Mean of empty slice



95% Bootstrap CI:
CI low : nan
CI high: nan
